### Import required libraries

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
import torch.nn.init as init
from torch.autograd import Variable
import datetime
from nab_dataset import NabDataset
from models.recurrent_models_pyramid import LSTMGenerator, LSTMDiscriminator
from models.soft_dtw_cuda import SoftDTW

import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import numpy as np

class ArgsTrn:
    workers=0
    batch_size=32
    epochs=20
    lr=0.0002
    cuda = True
    manualSeed=2
    window_size=60
    
opt_trn=ArgsTrn()

torch.manual_seed(opt_trn.manualSeed)
cudnn.benchmark = True

# location of datasets and category

end_name = 'ambient_temperature_system_failure.csv' # dataset name
data_file = 'data/realKnownCause/'+end_name # dataset category and dataset name
key = 'realKnownCause/'+end_name # This key is used for reading anomaly labels

# settings for data loader
class DataSettings:
    
    def __init__(self):
        self.BASE = 'NAB/'
        self.label_file = 'labels/combined_windows.json'
        self.data_file = data_file
        self.key = key
        self.train = True
        self.window_length = opt_trn.window_size
    
    
data_settings = DataSettings()

# define dataset object and data loader object for NAB dataset
dataset = NabDataset(data_settings=data_settings)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt_trn.batch_size,
                                         shuffle=True, num_workers=int(opt_trn.workers))

dataset.x.shape, dataset.y.shape # check the dataset shape

device = torch.device("cuda:0" if opt_trn.cuda else "cpu") # select the device
seq_len = dataset.window_length # sequence length is equal to the window length
in_dim = dataset.n_feature # input dimension is same as number of feature

# Create generator and discriminator models
netG = LSTMGenerator(in_dim=in_dim, out_dim=in_dim, device=device).to(device)
netD = LSTMDiscriminator(in_dim=in_dim, device=device).to(device)

print("|Generator Architecture|\n", netG)
print("|Discriminator Architecture|\n", netD)

# Setup loss function
criterion = nn.BCELoss().to(device)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt_trn.lr)
optimizerG = optim.Adam(netG.parameters(), lr=opt_trn.lr)

### Adversarial Training of Generator and Discriminator Models

In [ ]:
real_label = 1
fake_label = 0

loss_D = []
loss_G = []
loss_D_G_z = []

for epoch in range(opt_trn.epochs):
    for i, (x,y) in enumerate(dataloader, 0):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################

        #Train with real data
        netD.zero_grad()
        real = x.to(device)
        batch_size, seq_len = real.size(0), real.size(1)
        label = torch.full((batch_size, seq_len, 1), real_label, device=device)

        output,_ = netD.forward(real)
        errD_real = criterion(output, label.float())
        errD_real.backward()
        optimizerD.step()
        D_x = output.mean().item()
        
        #Train with fake data
        noise = Variable(init.normal(torch.Tensor(batch_size,seq_len,in_dim),mean=0,std=0.1)).cuda()
        fake,_ = netG.forward(noise)
        output,_ = netD.forward(fake.detach()) # detach causes gradient is no longer being computed or stored to save memeory
        label.fill_(fake_label)
        errD_fake = criterion(output, label.float())
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        noise = Variable(init.normal(torch.Tensor(batch_size,seq_len,in_dim),mean=0,std=0.1)).cuda()
        fake,_ = netG.forward(noise)
        label.fill_(real_label) 
        output,_ = netD.forward(fake)
        errG = criterion(output, label.float())
        errG.backward()
        optimizerG.step()
        D_G_z2 = output.mean().item()
        
    print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' 
          % (epoch+1, opt_trn.epochs, i, len(dataloader),
             errD.item(), errG.item(), D_x, D_G_z1, D_G_z2), end='')
    print()
    
    # add #
    loss_D.append(errD.item())
    loss_G.append(errG.item())
    mean_D_G_z = (D_G_z1+D_G_z2)/2
    loss_D_G_z.append(mean_D_G_z)
    
    
    if epoch > 0:
        if (errD.item() <= loss_D[epoch-1]*1.2) & (errG.item() <= loss_G[epoch-1]*1.2) & (errG.item() <= min(loss_G)) & (mean_D_G_z >= max(loss_D_G_z)):
            print('-> save model')
            torch.save(netD, 'NAB/model/netD_best.pth')
            torch.save(netG, 'NAB/model/netG_best.pth')
        else:
            print('-> Do not save model')
    else:
        print('-> save model')

## Anomaly Detection

#### Define basic settings for inverse mapping

In [ ]:
class ArgsTest:
    workers = 0
    batch_size = 1
    
opt_test=ArgsTest()    

# add #
netG = torch.load('NAB/model/netD_best.pth')
netD = torch.load('NAB/model/netD_best.pth')

generator = netG # changing reference variable 
discriminator = netD # changing reference variable 

# Define settings for loading data in evaluation mood
class TestDataSettings:
    
    def __init__(self):
        self.BASE = 'Nab/'
        self.label_file = 'labels/combined_windows.json'
        self.data_file = data_file
        self.key = key
        self.train = False
        self.window_length = opt_trn.window_size

# Lambda = 0.1 according to paper
# x is new data, G_z is closely regenerated data

def Anomaly_score(x, G_z, Lambda=0.1):
    residual_loss = torch.sum(torch.abs(x-G_z)) # Residual Loss
    
    # x_feature is a rich intermediate feature representation for real data x
    output, x_feature = discriminator(x.to(device)) 
    # G_z_feature is a rich intermediate feature representation for fake data G(z)
    output, G_z_feature = discriminator(G_z.to(device)) 
    
    discrimination_loss = torch.sum(torch.abs(x_feature-G_z_feature)) # Discrimination loss
    
    total_loss = (1-Lambda)*residual_loss.to(device) + Lambda*discrimination_loss
    return total_loss

def Generator_score(x, G_z, Lambda=0.9):
    criterion = nn.MSELoss()
    residual_loss = criterion(x, G_z)
    
    # x_feature is a rich intermediate feature representation for real data x
    output, x_feature = discriminator(x.to(device)) 
    # G_z_feature is a rich intermediate feature representation for fake data G(z)
    output, G_z_feature = discriminator(G_z.to(device)) 
    
    discrimination_loss = torch.sum(torch.abs(x_feature-G_z_feature)) # Discrimination loss
    total_loss = (1-Lambda)*residual_loss.to(device) + Lambda*discrimination_loss
    return total_loss

def Similarity_score(x, G_z):
    sdtw = SoftDTW(use_cuda=True, gamma=0.1)
    loss = sdtw(x, G_z)
    return loss.mean()

def replaced_minmax(real, z):
    path = dtw.warping_path(real, z)
    max_value = []
    min_value = []
    for i in range(len(path)):
        if path[i][0] == real.argmax():
            max_value.append(z[path[i][1]])
            
        if path[i][0] == real.argmin():
            min_value.append(z[path[i][1]])
    real[real.argmax()] = np.mean(max_value)
    real[real.argmin()] = np.mean(min_value)
    return real

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [ ]:
test_data_settings = TestDataSettings()

# define dataset object and data loader object in evaluation mood for NAB dataset
test_dataset = NabDataset(test_data_settings)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt_test.batch_size, 
                                         shuffle=False, num_workers=int(opt_test.workers))

test_dataset.x.shape, test_dataset.y.shape, test_dataset.data_len # check the dataset shape

In [ ]:
# Step 1. Anomaly detection for real data
step1_z_list = []
step1_gen_list = []
step1_real_list = []
step1_loss_list = []

for i, (x,y) in enumerate(test_dataloader):
    z = Variable(init.normal(torch.zeros(opt_test.batch_size,
                                     test_dataset.window_length, 
                                     test_dataset.n_feature),mean=0,std=0.1),requires_grad=True)
    #z = x
    z_optimizer = torch.optim.Adam([z],lr=1e-2)
    
    loss = None
    for j in range(50): # set your interation range
        gen_fake,_ = generator(z.cuda())
        loss = Anomaly_score(Variable(x).cuda(), gen_fake)
        loss.backward()
        z_optimizer.step()
    
    # add    
    step1_z_list.append(z)
    step1_gen_list.append(gen_fake)
    step1_real_list.append(Variable(x).cuda())
    step1_loss_list.append(loss) # Store the loss from the final iteration
    
    print('Index :', i, 'Anomaly :', y, 'loss={}'.format(loss))

### Visualise Anomaly Detection

In [ ]:
threshold = 5.9 
THRESHOLD = threshold

#TIME_STEPS = dataset.window_length
test_score_df = pd.DataFrame(index=range(test_dataset.data_len))
test_score_df['loss'] = [loss.item()/test_dataset.window_length for loss in step1_loss_list]
test_score_df['y'] = test_dataset.y
test_score_df['threshold'] = THRESHOLD
test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
test_score_df['t'] = [x[ArgsTrn.window_size-1].item() for x in test_dataset.x] 

plt.plot(test_score_df.index, test_score_df.loss, label='loss', color='k')
plt.plot(test_score_df.index, test_score_df.threshold, label='threshold', color='r')
plt.plot(test_score_df.index, test_score_df.y, label='anomaly', color='orange')
plt.xticks(rotation=0)
plt.legend();
plt.ylim([-1,7])

In [ ]:
anomalies = test_score_df[test_score_df.anomaly == True]

plt.plot(
  range(test_dataset.data_len), 
  test_score_df['t'], 
  label='value',
  color='black'
);

sns.scatterplot(
  # anomalies.index,
  anomalies.t,
  color=sns.color_palette()[3],
  s=52,
  label='predict_anomaly'
)

plt.plot(
  range(len(test_score_df['y'])),
  test_score_df['y'],
  label='anomaly',
  color = 'orange'
)

plt.xticks(rotation=0)
plt.legend();
plt.ylim([-4,4])

### Calculate the window-based anomalies

In [ ]:
start_end = []
state = 0
for idx in test_score_df.index:
    if state==0 and test_score_df.loc[idx, 'y']==1:
        state=1
        start = idx
    if state==1 and test_score_df.loc[idx, 'y']==0:
        state = 0
        end = idx
        start_end.append((start, end))

for s_e in start_end:
    if sum(test_score_df[s_e[0]:s_e[1]+1]['anomaly'])>0:
        for i in range(s_e[0], s_e[1]+1):
            test_score_df.loc[i, 'anomaly'] = 1
            
actual = np.array(test_score_df['y'])
predicted = np.array([int(a) for a in test_score_df['anomaly']])

import numpy as np
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

predicted = np.array(predicted)
actual = np.array(actual)

tp = np.count_nonzero(predicted * actual)
tn = np.count_nonzero((predicted - 1) * (actual - 1))
fp = np.count_nonzero(predicted * (actual - 1))
fn = np.count_nonzero((predicted - 1) * actual)

print('True Positive\t', tp)
print('True Negative\t', tn)
print('False Positive\t', fp)
print('False Negative\t', fn)

accuracy = (tp + tn) / (tp + fp + fn + tn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
fmeasure = (2 * precision * recall) / (precision + recall)
cohen_kappa_score = cohen_kappa_score(predicted, actual)
false_positive_rate, true_positive_rate, thresholds = roc_curve(actual, predicted)
auc_val = auc(false_positive_rate, true_positive_rate)
roc_auc_val = roc_auc_score(actual, predicted)

print('Accuracy\t', accuracy)
print('Precision\t', precision)
print('Recall\t', recall)
print('f-measure\t', fmeasure)
print('cohen_kappa_score\t', cohen_kappa_score)
print('auc\t', auc_val)
print('roc_auc\t', roc_auc_val)

In [ ]:
anomalies = test_score_df[test_score_df.anomaly == True]

plt.plot(
  range(test_dataset.data_len), 
  test_score_df['t'], 
  label='value',
  color='black'
);

sns.scatterplot(
  # anomalies.index,
  anomalies.t,
  color=sns.color_palette()[3],
  s=52,
  label='predict_anomaly'
)

plt.plot(
  range(len(test_score_df['y'])),
  test_score_df['y'],
  label='anomaly',
  color = 'orange'
)

plt.xticks(rotation=0)
plt.legend();
plt.ylim([-4,4])

In [ ]:
# define the anomaly index 
raw_anomaly_index = []
for i in range(len(predicted)):
    if predicted[i] == 1:
        # print(i, predicted[i])
        raw_anomaly_index.append(i)
raw_anomaly_index = np.array(raw_anomaly_index)
raw_anomaly_index

### Genenrate entire data

In [ ]:
step2_z_list = []
step2_gen_list = []
step2_real_list = []
step2_loss_list = []

total_mean = []
total_std = []

for i, (x,y) in enumerate(test_dataloader):

    win_mean = np.mean(Variable(x)[-1,:,-1].cpu().detach().numpy())
    win_std = np.std(Variable(x)[-1,:,-1].cpu().detach().numpy())
    
    total_mean.append(win_mean) 
    total_std.append(win_std)
    
    z = Variable(init.normal(torch.zeros(opt_test.batch_size,
                                     test_dataset.window_length, 
                                     test_dataset.n_feature),
                                     mean=win_mean,std=win_std),requires_grad=True)

    z_optimizer = torch.optim.Adam([z],lr=5e-3) # 1e-2, 5e-3
    
    loss = None
    
    for j in range(50): # set your interation range
        gen_fake,_ = generator(z.cuda())
        loss = Generator_score(Variable(x).cuda(), gen_fake) + Similarity_score(Variable(x).cuda(), z.cuda())
        loss.backward()
        z_optimizer.step()

    # add    
    step2_z_list.append(z)
    step2_gen_list.append(gen_fake)
    step2_real_list.append(Variable(x).cuda())
    step2_loss_list.append(loss) # Store the loss from the final iteration

    print('Generate process - Index :', i, 'Anomaly :', y, 'loss={}'.format(loss))

In [ ]:
test_dataset = NabDataset(test_data_settings)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt_test.batch_size, 
                                         shuffle=False, num_workers=int(opt_test.workers))

replace_dataset = NabDataset(test_data_settings)

In [ ]:
total_idx = np.array(list(range(replace_dataset.x.shape[0])))
normal_idx = np.delete(total_idx, [raw_anomaly_index])

# Data Correction

## Proposed Method

In [ ]:
###################################################################
# Step 1. Anomaly detection for total data
###################################################################
from datetime import datetime

ano_idx = raw_anomaly_index
epochs = 30

step3_z_list = step2_z_list
step3_gen_list = step2_gen_list
step3_real_list = step2_real_list
step3_loss_list = step2_loss_list

mean_window_raw = np.zeros(shape=(len(replace_dataset),))
std_window_raw = np.zeros(shape=(len(replace_dataset),))
for i in range(epochs):
    epoch = i
    start_time = datetime.now()
    print('Replacement process - epoch: {}/{} Start time: {}'.format(i+1, epochs, start_time))
    replace_dataloader = torch.utils.data.DataLoader(replace_dataset, batch_size=opt_test.batch_size, 
                                         shuffle=False, num_workers=int(opt_test.workers))
    
    ###################################################################
    # Step 2. Generate data
    ###################################################################
    for i, (x,y) in enumerate(replace_dataloader):
        # generate about only anomaly data
        if i in ano_idx:
            # define mean and std
            mean_window = np.mean(Variable(x)[-1,:,-1].cpu().detach().numpy())
            std_window = np.std(Variable(x)[-1,:,-1].cpu().detach().numpy())
            
            # 초기 평균 및 표준편차 저장
            if epoch == 0:
                mean_window_raw[i] = mean_window
                std_window_raw[i] = std_window
                
            if epoch % 5 == 0:
                mean_window = mean_window_raw[i]
                std_window = std_window_raw[i]      
            
            nm_idx = find_nearest(normal_idx, i)
            new_mean = total_mean[nm_idx]
            new_std = total_std[nm_idx]/2
            
            z = Variable(init.normal_(torch.zeros(opt_test.batch_size,
                                                  replace_dataset.window_length, 
                                                  replace_dataset.n_feature),
                                                  # mean=mean_window,std=std_window),requires_grad=True)
                                                  # mean=total_mean[nm_idx],std=total_std[nm_idx]),requires_grad=True)
                                                  mean=new_mean, std=new_std), requires_grad=True)

            z_optimizer = torch.optim.Adam([z],lr=5e-3) # 1e-2, 5e-3

            loss = None
            for j in range(100): # set your interation range
                gen_fake,_ = generator(z.cuda())
                loss = Anomaly_score(Variable(x).cuda(), gen_fake) + Similarity_score(step2_z_list[nm_idx].cuda(), z.cuda()) + Similarity_score(Variable(x).cuda(), z.cuda())
                loss.mean().backward()
                z_optimizer.step()

            # add        
            step3_z_list[i] = z
            step3_gen_list[i] = gen_fake
            step3_real_list[i] = Variable(x).cuda()
            step3_loss_list[i] = loss # Store the loss from the final iteration

            # print('Generate process - Index :', i, 'Anomaly :', y, 'loss={}'.format(loss))
            
    ###################################################################
    # Step 3. Replace real data with generated data
    ###################################################################
    ## step 1. replace generate data with real data
    replaced_data = []
    for i in ano_idx:
        real = step3_real_list[i][-1,:,-1].cpu().detach().numpy()
        z = step3_z_list[i][-1,:,-1].cpu().detach().numpy()
        replaced_data.append(replaced_minmax(real, z))

    ## step 2. mapping replaced data to real data
    for i, j in enumerate(ano_idx):
        replace_dataset.x[j] = torch.from_numpy(replaced_data[i].reshape(60,1))
        
    ###################################################################
    # Step 4. Anomaly detection for anomaly data
    ###################################################################
    step4_loss_list = step1_loss_list
    for i, (x,y) in enumerate(replace_dataloader):
        if i in ano_idx:
            z = Variable(init.normal_(torch.zeros(opt_test.batch_size,
                                             test_dataset.window_length, 
                                             test_dataset.n_feature),mean=0,std=0.1),requires_grad=True)
            #z = x
            z_optimizer = torch.optim.Adam([z],lr=1e-2)

            loss = None
            for j in range(50): # set your interation range
                gen_fake,_ = generator(z.cuda())
                loss = Anomaly_score(Variable(x).cuda(), gen_fake)
                loss.backward()
                z_optimizer.step()

            step4_loss_list[i] = loss # Store the loss from the final iteration
        
    THRESHOLD = threshold # Anomaly score threshold for an instance to be considered as anomaly 

    #TIME_STEPS = dataset.window_length
    test_score_df_after = pd.DataFrame(index=range(replace_dataset.data_len))
    test_score_df_after['loss'] = [loss.item()/replace_dataset.window_length for loss in step4_loss_list]
    test_score_df_after['y'] = replace_dataset.y
    test_score_df_after['threshold'] = THRESHOLD
    test_score_df_after['anomaly'] = test_score_df_after.loss > test_score_df_after.threshold
    test_score_df_after['t'] = [x[ArgsTrn.window_size-1].item() for x in replace_dataset.x]
    
    start_end = []
    state = 0
    for idx in test_score_df_after.index:
        if state==0 and test_score_df_after.loc[idx, 'y']==1:
            state=1
            start = idx
        if state==1 and test_score_df_after.loc[idx, 'y']==0:
            state = 0
            end = idx
            start_end.append((start, end))

    for s_e in start_end:
        if sum(test_score_df_after[s_e[0]:s_e[1]+1]['anomaly'])>0:
            for i in range(s_e[0], s_e[1]+1):
                test_score_df_after.loc[i, 'anomaly'] = 1

    actual = np.array(test_score_df_after['y'])
    predicted = np.array([int(a) for a in test_score_df_after['anomaly']])
   
    # define the anomaly index 
    ano_idx = []
    for i in range(len(predicted)):
        if predicted[i] == 1:
            ano_idx.append(i)
    ano_idx = np.array(ano_idx)
    ano_idx
    
    if len(ano_idx) < 1:
        print('Finish Replacement process')
        break